In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import streamlit as st
import altair as alt
import plotly.graph_objects as go
import google.generativeai as genai
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import base64
from pathlib import Path

BASE_DIR = Path(__file__).resolve().parent
csv_path = BASE_DIR / "crop_yield_short.csv"
image_pathh = BASE_DIR / "background2.jpg"
text_path = BASE_DIR / "crop_yield_shorter.txt"

st.set_page_config(page_title="Yield Wizzard 3000", layout="centered")

def set_bg_with_overlay(image_path):
    with open(image_path, "rb") as image_file:
        encoded = base64.b64encode(image_file.read()).decode()
    css = f"""
    <style>
    .stApp {{
        background: linear-gradient(
            rgba(0, 0, 0, 0.7), 
            rgba(0, 0, 0, 0.7)
        ), url("data:image/jpg;base64,{encoded}"); 
        background-size: cover;
        background-position: center;
    }}

    .title {{
        text-align: center;
        font-size: 2.5em;
        font-weight: bold;
        margin: 20px 0;
        color: white;
    }}
    
    /* Centered tab container */
    div[data-testid="stTabs"] {{
        display: flex;
        justify-content: center;
    }}
    
    /* Centered tab list */
    .stTabs [data-baseweb="tab-list"] {{
        gap: 10px;
        padding: 10px 0;
        display: flex;
        justify-content: center;
    }}
    
    /* Tab styling */
    .stTabs [data-baseweb="tab"] {{
        padding: 10px 20px;
        border-radius: 20px !important;
        background-color: white !important;
        color: #333 !important;
        font-weight: bold;
        transition: all 0.3s ease;
        border: none !important;
        box-shadow: 0 2px 5px rgba(0,0,0,0.1);
    }}
    
    .stTabs [data-baseweb="tab"]:hover {{
        background-color: #f0f0f0 !important;
        transform: translateY(-2px);
        box-shadow: 0 4px 8px rgba(0,0,0,0.15);
    }}
    
    .stTabs [aria-selected="true"] {{
        background-color: #4a8fe7 !important;
        color: white !important;
        box-shadow: 0 4px 8px rgba(74, 143, 231, 0.3);
    }}
    
    /* Remove the bottom border line */
    .stTabs [data-baseweb="tab-highlight"] {{
        background-color: transparent !important;
    }}
    </style>
    """
    st.markdown(css, unsafe_allow_html=True)

set_bg_with_overlay(image_pathh)

df_plots = pd.read_csv(csv_path)
df_model = pd.read_csv(csv_path)

if 'api_key_valid' not in st.session_state:
    st.session_state.api_key_valid = False

# Loop until valid API key is entered
if not st.session_state.api_key_valid:
    st.sidebar.header("🔐 API Key Setup")
    user_api_key = st.sidebar.text_input("Enter your Gemini API Key", type="password")

    if user_api_key:
        try:
            genai.configure(api_key=user_api_key)
            model = genai.GenerativeModel('gemini-1.5-flash')
            _ = model.generate_content("Test")  # Dummy request

            # If successful
            st.session_state.api_key_valid = True
            st.success("API key is valid!")

            # Hide sidebar after success
            hide_sidebar = """
            <style>
                [data-testid="stSidebar"] {
                    display: none;
                }
                [data-testid="collapsedControl"] {
                    display: none;
                }
            </style>
            """
            st.markdown(hide_sidebar, unsafe_allow_html=True)

        except Exception:
            st.sidebar.error("Invalid API key. Please enter a valid key.")
            st.stop()
    else:
        st.warning("Please enter your Gemini API key to proceed.")
        st.stop()

graph_type = ['Bar Graph', 'Scatter plot', 'Box Plot', 'Line Chart']

def make_graph(typ):
    if typ == graph_type[3]:
        grouped = df_plots.groupby(['Crop', 'Region']).size().reset_index(name='Frequency')
        color_scale = alt.Scale(domain=["North", "South", "East", "West"], range=["#1f77b4", "#2ca02c", "#d62728", "#ffc107"])
        chart = alt.Chart(grouped).mark_line(point=True).encode(
            x='Crop:N',
            y='Frequency:Q',
            color=alt.Color('Region:N', scale=color_scale, legend=alt.Legend(title="Region"))
        ).properties(width=700, height=400, title="Crop Frequency by Region (Custom Colors)")
        st.altair_chart(chart, use_container_width=True)
    elif typ == graph_type[0]:
        avg_yield = df_plots.groupby("Crop")["Yield_tons_per_hectare"].mean().reset_index()
        colors = ['#FFFF00', '#FFD700', '#FFA500', '#FF7F00', '#FF4500', '#FF0000']
        unique_crops = avg_yield['Crop'].unique()
        color_map = {crop: colors[i % len(colors)] for i, crop in enumerate(unique_crops)}
        avg_yield['Color'] = avg_yield['Crop'].map(color_map)
        chart = alt.Chart(avg_yield).mark_bar().encode(
            x=alt.X("Crop:N", title="Crop"),
            y=alt.Y("Yield_tons_per_hectare:Q", title="Average Yield (Ton per Hectare)"),
            color=alt.Color("Crop:N", scale=alt.Scale(domain=list(color_map.keys()), range=list(color_map.values())), legend=None)
        ).properties(title="Average Yield per Crop", width=100, height=400)
        st.altair_chart(chart, use_container_width=True)
    elif typ == graph_type[1]:
        scatter = alt.Chart(df_plots).mark_circle(size=100).encode(
            x=alt.X("Rainfall_mm", title="Rainfall (mm)"),
            y=alt.Y("Yield_tons_per_hectare", title="Yield (tons/hectare)"),
            color=alt.Color("Crop", legend=alt.Legend(title="Crop")),
            tooltip=["Crop", "Rainfall_mm", "Yield_tons_per_hectare"]
        ).properties(title="Yield vs. Rainfall Scatter Plot", width=600, height=400).interactive()
        st.altair_chart(scatter, use_container_width=True)
    elif typ == graph_type[2]:
        color_scale = alt.Scale(domain=["Cloudy", "Rainy", "Sunny"], range=["#a6cee3", "#e31a1c", "#1b9e77"])
        chart = alt.Chart(df_plots).mark_boxplot(extent='min-max').encode(
            x=alt.X('Weather_Condition:N', title='Weather Condition'),
            y=alt.Y('Yield_tons_per_hectare:Q', title='Yield (tons/hectare)'),
            color=alt.Color('Weather_Condition:N', scale=color_scale, legend=alt.Legend(title="Weather"))
        ).properties(width=1000, height=400, title="Yield vs. Weather Condition (Box Plot)")
        st.altair_chart(chart, use_container_width=True)

# Set the background image and title
st.markdown(f'<div class="title">Yield Wizzard 3000</div>', unsafe_allow_html=True)

# Create centered tabs
col1, col2, col3 = st.columns([1,2,1])
with col2:
    tab1, tab2, tab3, tab4, tab5 = st.tabs(["Data Frame", "Graph Options", "Summary and Stats", "Model Training", "About developers"])

    with tab1:
        st.subheader("Data")
        st.dataframe(df_plots)

    with tab2:
        st.subheader('Graph options')
        choice_graph_type = st.selectbox("Choose Graph Type", list(graph_type))
        make_graph(choice_graph_type)


    with tab3:
        st.subheader('Get Statistical analysis with the Chat Wizzard')

        with st.form("analysis_form"):
            st.subheader("\U0001F4C8 Select Graph for Analysis")
            graph_target = st.selectbox(
                "Which relationship or comparison do you want to analyze?",
                [
                    "Crop vs Region",
                    "Rainfall vs Yield",
                    "Temperature vs Yield",
                    "Soil Type vs Yield",
                    "Weather Condition vs Crop",
                ]
            )

            st.subheader("\U0001F4CA Chart Types Used")
            chart_bar = st.checkbox("Bar Chart")
            chart_line = st.checkbox("Line Chart")
            chart_box = st.checkbox("Box Plot")
            chart_scatter = st.checkbox("Scatter Plot")

            st.subheader("❓ Custom Questions for Analysis")
            question_1 = st.text_input("Question 1", placeholder="e.g., Which region gives highest yield for Cotton?")
            question_2 = st.text_input("Question 2", placeholder="e.g., Is higher rainfall always better for crops?")
            question_3 = st.text_input("Question 3", placeholder="e.g., Do Peaty soils outperform Loam in wheat growth?")

            st.subheader("\U0001F529 Optional Data Filters")
            temp_range = st.slider("Temperature Range (°C)", 10, 45, (15, 35))
            rainfall_range = st.slider("Rainfall Range (mm)", 0, 1200, (100, 1000))

            submitted = st.form_submit_button("Ask Wizzi")

            if submitted:
                chart_types = []
                if chart_bar: chart_types.append("Bar Chart")
                if chart_line: chart_types.append("Line Chart")
                if chart_box: chart_types.append("Box Plot")
                if chart_scatter: chart_types.append("Scatter Plot")
                charts_used = ", ".join(chart_types) if chart_types else "No charts selected"

                final_prompt = f"""
    You are a data analyst. You are given a dataset and a visualization showing {graph_target}. Analyze the data using statistical reasoning.

    Charts Used:
    {charts_used}

    Analysis Tasks:
    1. Identify patterns and trends in the relationship between {graph_target}.
    2. Detect any outliers or unusual values.
    3. Comment on potential correlations or regional/categorical performance.
    4. Answer these user questions:
    - {question_1 or '[User Question 1]'}
    - {question_2 or '[User Question 2]'}
    - {question_3 or '[User Question 3]'}

    User-specified Filters:
    - Temperature Range: {temp_range[0]}°C to {temp_range[1]}°C
    - Rainfall Range: {rainfall_range[0]} mm to {rainfall_range[1]} mm

    Instructions:
    Respond in paragraphs or bullet points. Use basic statistical terms (mean, range, correlation) where applicable.
    Use a maximum of 2 paragraphs with up to 5 lines each. Each paragraph can replace 5 bullet points and vice versa. at the end answer the user asked question seperatly. make the paragraphs consistent with 5 lines max.
    """

                with open(text_path, "r") as f:
                    file_text = f.read()

                combined_prompt = final_prompt + "\n\n\U0001F4C4 Dataset Snapshot:\n" + file_text

                model = genai.GenerativeModel('gemini-1.5-flash')
                response = model.generate_content(combined_prompt)

                st.subheader("\U0001F4CB AI Analysis Output")
                st.write(response.text)

df_model['Region']= df_model['Region'].replace({'North':1.0, 'South':2.0, 'East':3.0, 'West':4.0})
df_model['Soil_Type']= df_model['Soil_Type'].replace({'Sandy':1.0, 'Clay':2.0, 'Loam':3.0, 'Silt':4.0, 'Peaty':5.0, 'Chalky':6.0})
df_model['Crop']=df_model['Crop'].replace({'Cotton':1.0, 'Rice':2.0, 'Barley':3.0, 'Soybean':4.0, 'Wheat':5.0, 'Maize':6.0})
df_model['Irrigation_Used']=df_model['Irrigation_Used'].replace({True:1.0, False:0.0})
df_model['Fertilizer_Used']=df_model['Fertilizer_Used'].replace({True:1.0, False:0.0})
df_model['Weather_Condition']=df_model['Weather_Condition'].replace({'Cloudy':1.0, 'Rainy':2.0, 'Sunny':3.0})
df_model['Days_to_Harvest']=df_model['Days_to_Harvest'].astype(float)

x=df_model[['Region', 'Soil_Type', 'Crop', 'Rainfall_mm', 'Temperature_Celsius',
       'Fertilizer_Used', 'Irrigation_Used', 'Weather_Condition',
       'Days_to_Harvest']]
y=df_model['Yield_tons_per_hectare']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(x_train, y_train)

y_pred= model.predict(x_test)

MSE= mean_squared_error(y_test, y_pred)
R2= r2_score(y_test, y_pred)

    
with tab4:
    st.subheader("🧠 Learning Model Results")
    st.write("**Learning Model used:** Linear Regression")
    st.write(f"**MSE:** {MSE}")
    st.write(f"**R² Score:** {R2}")
    
with tab5:
    st.subheader("👨‍💻 About the Developers")
    st.markdown("""
        ### Yield Wizzard 3000
        Developed as a part of the Artificial Intelligence Lab Project at GIK Institute.

        **Team Members:**
        - **Muhammad Ibrahim Umar** (2024413) 
        - **Muhammad Shahnawaz**    (2024464)

        **Tools & Technologies:**
        - Python, Pandas, NumPy, Streamlit
        - Machine Learning (Linear Regression)
        - Altair, Plotly, Seaborn
        - Gemini API for LLM-driven analysis

        **Project Objective:**
        To visualize and predict agricultural crop yield based on environmental and regional data.
        """, unsafe_allow_html=True)

d:\Python\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name '__file__' is not defined

In [ ]:
pip install matplotlib

In [ ]:
pip install seaborn

In [ ]:
pip install streamlit

In [ ]:
pip install altair

In [ ]:
pip install plotly

In [ ]:
pip install pathlib

In [ ]:
pip install base64

In [ ]:
"""
Advanced Car Racing Simulator Game using Pygame
- 2D top-down perspective racing on a curved track
- Smooth car physics: acceleration, braking, turning, friction
- Lap timing system
- High graphical quality for 2D pygame standards
- Keyboard controls: arrows or WASD

Ensure pygame is installed: pip install pygame
Run the game: python car_racing_simulator.py
"""

import pygame
import math
import time
import sys

# Initialize pygame
pygame.init()
pygame.mixer.init()

# Constants
WIDTH, HEIGHT = 800, 600  # Window size
FPS = 60

# Colors
WHITE = (255, 255, 255)
GRAY = (100, 100, 100)
DARKGRAY = (50, 50, 50)
GREEN = (0, 200, 0)
RED = (200, 0, 0)
BLACK = (0, 0, 0)
YELLOW = (255, 255, 0)
BLUE = (70, 130, 180)

# Set up display
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Advanced Car Racing Simulator")

# Load font
font = pygame.font.SysFont("Arial", 20, bold=True)
big_font = pygame.font.SysFont("Arial", 40, bold=True)

# Clock
clock = pygame.time.Clock()

# Helper Functions
def blit_rotate_center(surf, image, top_left, angle):
    """Rotate an image and blit it to the surface with its center maintained."""
    rotated_image = pygame.transform.rotozoom(image, -angle, 1)  # rotozoom for better scaling
    new_rect = rotated_image.get_rect(center=image.get_rect(topleft=top_left).center)
    surf.blit(rotated_image, new_rect.topleft)
    return new_rect

def load_image(path, scale=None):
    """Load an image and optionally scale it."""
    image = pygame.image.load(path).convert_alpha()
    if scale:
        image = pygame.transform.smoothscale(image, scale)
    return image

# Car class
class Car:
    def __init__(self, x, y):
        # Load car image (simple sports car)
        self.original_image = load_image("https://images.pexels.com/photos/170811/pexels-photo-170811.jpeg?auto=compress&cs=tinysrgb&h=64", scale=(64, 32))
        # Since we cannot load external in pygame, let's create a car surface programmatically
        self.original_image = pygame.Surface((64, 32), pygame.SRCALPHA)
        pygame.draw.polygon(self.original_image, BLUE, [(0, 0), (64, 8), (64, 24), (0, 32)])
        pygame.draw.rect(self.original_image, WHITE, (0, 8, 64, 16), 2)
        # Car properties
        self.x = x
        self.y = y
        self.angle = 0  # Car facing angle in degrees
        self.speed = 0
        self.max_speed = 7
        self.acceleration = 0.15
        self.deceleration = 0.3
        self.friction = 0.05
        self.turn_speed = 4
        self.length = 64
        self.width = 32

    def update(self, keys):
        # Controls for acceleration
        if keys[pygame.K_UP] or keys[pygame.K_w]:
            self.speed += self.acceleration
        elif keys[pygame.K_DOWN] or keys[pygame.K_s]:
            self.speed -= self.deceleration
        else:
            # Apply friction when not accelerating
            if self.speed > 0:
                self.speed -= self.friction
            elif self.speed < 0:
                self.speed += self.friction

        # Clamp speed
        if self.speed > self.max_speed:
            self.speed = self.max_speed
        if self.speed < -self.max_speed / 2:
            self.speed = -self.max_speed / 2  # reverse slower

        # Turning
        if (keys[pygame.K_LEFT] or keys[pygame.K_a]) and self.speed != 0:
            turn_direction = 1 if self.speed > 0 else -1
            self.angle += self.turn_speed * turn_direction

        if (keys[pygame.K_RIGHT] or keys[pygame.K_d]) and self.speed != 0:
            turn_direction = -1 if self.speed > 0 else 1
            self.angle += self.turn_speed * turn_direction

        # Update position based on speed and angle
        rad = math.radians(self.angle)
        self.x += -math.sin(rad) * self.speed
        self.y += -math.cos(rad) * self.speed

    def draw(self, surface):
        blit_rotate_center(surface, self.original_image, (self.x - self.length // 2, self.y - self.width // 2), self.angle)


# Track class
class Track:
    def __init__(self):
        # Create a surface to draw the track
        self.surface = pygame.Surface((WIDTH, HEIGHT))
        self.surface.fill(GREEN)  # Grass background

        # Track boundaries (rectangles)
        # We'll paint the track as gray roads with white lane markings
        self.track_color = GRAY
        self.road_width = 180

        # Define the center path of the track as a series of points (loop)
        self.center_path = [
            (WIDTH // 2, HEIGHT // 4), (WIDTH // 2 + 160, HEIGHT // 2 - 100),
            (WIDTH // 2 + 180, HEIGHT // 2 + 50), (WIDTH // 2 + 140, HEIGHT // 2 + 150),
            (WIDTH // 2, HEIGHT // 2 + 210), (WIDTH // 2 - 130, HEIGHT // 2 + 180),
            (WIDTH // 2 - 160, HEIGHT // 2 + 100), (WIDTH // 2 - 110, HEIGHT // 2 - 30)
        ]
        # Make it loopable by adding first point again at end for curves
        self.center_path.append(self.center_path[0])

        # Pre-render track for performance
        self._draw_track()

        # Define start/finish line rect for lap timing
        self.finish_line = pygame.Rect(WIDTH // 2 - 5, HEIGHT // 4 - self.road_width // 2, 10, self.road_width)
        self.finish_line_color = RED

    def _draw_track(self):
        # Clear surface bg to grass green
        self.surface.fill(GREEN)

        # Draw the curved track using polygon segments between points
        points_left = []
        points_right = []

        for i in range(len(self.center_path) - 1):
            cx, cy = self.center_path[i]
            nx, ny = self.center_path[i + 1]
            # calculate direction perpendicular to path segment
            dx, dy = nx - cx, ny - cy
            length = math.hypot(dx, dy)
            if length == 0:
                length = 1
            dx /= length
            dy /= length
            # perpendicular vector (to the left)
            plx = -dy * (self.road_width / 2)
            ply = dx * (self.road_width / 2)

            points_left.append((cx + plx, cy + ply))
            points_right.append((cx - plx, cy - ply))

        # Add last point perpendicular offsets for closing polygon
        cx, cy = self.center_path[-1]
        nx, ny = self.center_path[0]
        dx, dy = nx - cx, ny - cy
        length = math.hypot(dx, dy)
        if length == 0:
            length = 1
        dx /= length
        dy /= length
        plx = -dy * (self.road_width / 2)
        ply = dx * (self.road_width / 2)
        points_left.append((cx + plx, cy + ply))
        points_right.append((cx - plx, cy - ply))

        # Full polygon points for track road
        track_poly = points_left + points_right[::-1]

        # Draw track road
        pygame.draw.polygon(self.surface, self.track_color, track_poly)

        # Draw lane separator lines on track center path
        for i in range(len(self.center_path) - 1):
            start = self.center_path[i]
            end = self.center_path[i + 1]
            # Draw dashed lines for lane markings
            self._draw_dashed_line(self.surface, WHITE, start, end, dash_length=15, space_length=10, width=3)

        # Draw finish line
        pygame.draw.rect(self.surface, self.finish_line_color, self.finish_line)

    def _draw_dashed_line(self, surface, color, start_pos, end_pos, dash_length=10, space_length=5, width=2):
        x1, y1 = start_pos
        x2, y2 = end_pos
        length = math.hypot(x2 - x1, y2 - y1)
        dash_count = int(length / (dash_length + space_length))
        dash_dx = (x2 - x1) / length * dash_length
        dash_dy = (y2 - y1) / length * dash_length
        space_dx = (x2 - x1) / length * space_length
        space_dy = (y2 - y1) / length * space_length

        for i in range(dash_count):
            start_x = x1 + (dash_dx + space_dx) * i
            start_y = y1 + (dash_dy + space_dy) * i
            end_x = start_x + dash_dx
            end_y = start_y + dash_dy
            pygame.draw.line(surface, color, (start_x, start_y), (end_x, end_y), width)

    def draw(self, surface):
        surface.blit(self.surface, (0, 0))

    def is_on_track(self, x, y):
        # Check pixel color at position to detect if on track pavement or off track (grass)
        try:
            color = self.surface.get_at((int(x), int(y)))[:3]
            # Track pavement is gray (self.track_color)
            # Using a tolerance because of anti-aliasing
            def color_close(c1, c2, tol=30):
                return all(abs(c1[i] - c2[i]) < tol for i in range(3))
            if color_close(color, self.track_color) or color_close(color, WHITE):
                return True
            else:
                return False
        except IndexError:
            return False

    def check_finish_line_cross(self, rect, prev_rect):
        # Check if the car crossed the finish line from previous position to current
        if self.finish_line.colliderect(rect):
            if not self.finish_line.colliderect(prev_rect):
                return True
        return False


# Main Game class
class Game:
    def __init__(self):
        self.track = Track()
        # Start car at starting position on track center
        start_x, start_y = self.track.center_path[0]
        self.car = Car(start_x, start_y + 50)
        self.start_time = None
        self.lap_time = 0
        self.best_lap_time = None
        self.laps_completed = 0
        self.last_finish_cross = False
        self.prev_car_rect = None
        self.game_over = False

        # Sounds (basic)
        try:
            self.sound_accel = pygame.mixer.Sound(pygame.mixer.Sound(file=None))
            self.sound_brake = pygame.mixer.Sound(pygame.mixer.Sound(file=None))
        except:
            self.sound_accel = None
            self.sound_brake = None

    def display_text(self, surface, text, x, y, color=WHITE, font_obj=None):
        font_obj = font_obj or font
        textobj = font_obj.render(text, True, color)
        surface.blit(textobj, (x, y))

    def update(self):
        keys = pygame.key.get_pressed()
        self.car.update(keys)

        # Keep previous rect for finish line checking
        car_rect = pygame.Rect(self.car.x - self.car.length//2, self.car.y - self.car.width//2, self.car.length, self.car.width)
        if self.prev_car_rect is None:
            self.prev_car_rect = car_rect.copy()

        # Check if on track, if not reduce speed gradually
        if not self.track.is_on_track(self.car.x, self.car.y):
            self.car.speed *= 0.95  # Slow down if off track

        # Handle lap timing and finish line crossing
        crossed_finish = self.track.check_finish_line_cross(car_rect, self.prev_car_rect)
        if crossed_finish and self.car.speed > 0:
            now = time.time()
            if self.start_time is None:
                self.start_time = now
            else:
                lap = now - self.start_time
                if self.best_lap_time is None or lap < self.best_lap_time:
                    self.best_lap_time = lap
                self.lap_time = lap
                self.start_time = now
                self.laps_completed += 1

        self.prev_car_rect = car_rect.copy()

    def draw(self, surface):
        surface.fill(GREEN)
        self.track.draw(surface)
        self.car.draw(surface)

        # Display speed
        speed_text = f"Speed: {int(self.car.speed * 20)} km/h"
        self.display_text(surface, speed_text, 10, 10)

        # Display laps and times
        laps_text = f"Laps Completed: {self.laps_completed}"
        self.display_text(surface, laps_text, 10, 40)

        if self.lap_time > 0:
            lap_text = f"Last Lap Time: {self.lap_time:.2f} s"
            self.display_text(surface, lap_text, 10, 70)

        if self.best_lap_time is not None:
            best_text = f"Best Lap Time: {self.best_lap_time:.2f} s"
            self.display_text(surface, best_text, 10, 100)

        # Instructions
        instructions = "Controls: Arrow Keys / WASD to drive | ESC to quit"
        self.display_text(surface, instructions, 10, HEIGHT - 30, YELLOW)

    def run(self):
        running = True
        while running:
            dt = clock.tick(FPS)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        running = False

            self.update()
            self.draw(screen)
            pygame.display.flip()

        pygame.quit()
        sys.exit()

# Run the game
if __name__ == "__main__":
    game = Game()
    game.run()


AttributeError: 'Track' object has no attribute 'finish_line_color'

: 

In [3]:
pip install pygame


  Using cached pygame-2.6.1-cp313-cp313-win_amd64.whl.metadata (13 kB)
Using cached pygame-2.6.1-cp313-cp313-win_amd64.whl (10.6 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install math

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement math (from versions: none)
ERROR: No matching distribution found for math


In [5]:
pip install time

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time
